In [11]:
# !pip install html2text requests_html
# !pip install -U llama-index
# !pip install -U openai
# !pip install pyyaml requests transformers
# !pip3 install torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 4.3 MB/s eta 0:00:0000:0100:01
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached MarkupSafe-2.1.3-cp39-cp39-macosx_10_9_universal2.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 7.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 4.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 7.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.8 MB/s eta 0:00:00a 0:00:01
Using cached MarkupSafe-2.1.3-cp39-cp39-macosx_10_9_universal2.whl (17 kB)


In [1]:
import sys
import logging
import html2text
import nest_asyncio
from typing import List
import yaml, os, openai, textwrap
from llama_index.llms import OpenAI
from requests_html import HTMLSession
from llama_index.llms import AzureOpenAI
from llama_index.llm_predictor import LLMPredictor
from llama_index import set_global_service_context
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index import StorageContext, load_index_from_storage
from llama_index import ServiceContext, SimpleDirectoryReader, TreeIndex, VectorStoreIndex

In [2]:
import llama_index, openai
print("llama-index version:", llama_index.__version__)
print("openai version:", openai.__version__)

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

llama-index version: 0.9.5
openai version: 1.3.4


In [3]:
with open('/Users/1zuu/Desktop/LLM RESEARCH/awesome-llm-projects/cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

# os.environ["OPENAI_API_TYPE"] = credentials['AD_OPENAI_API_TYPE']
# os.environ["OPENAI_API_VERSION"] = credentials['AD_OPENAI_API_VERSION']
# os.environ["OPENAI_API_BASE"] = credentials['AD_OPENAI_API_BASE']
# os.environ["OPENAI_API_KEY"] = credentials['AD_OPENAI_API_KEY']

In [4]:
chat_llm = LLMPredictor(
                        llm=AzureOpenAI(
                                        deployment_name=credentials['AD_DEPLOYMENT_ID'],
                                        model=credentials['AD_ENGINE'],
                                        api_key=credentials['AD_OPENAI_API_KEY'],
                                        api_version=credentials['AD_OPENAI_API_VERSION'],
                                        azure_endpoint=credentials['AD_OPENAI_API_BASE']
                                        )
                        )
embedding_llm = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

service_context = ServiceContext.from_defaults(
                                                llm_predictor=chat_llm,
                                                embed_model=embedding_llm,
                                                )

set_global_service_context(service_context)

/Users/1zuu/miniforge3/envs/llamaindex/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
def download_and_save_in_markdown(
                                url: str, 
                                dir_path: str
                                ) -> None:
    """Download the HTML content from the web page and save it as a markdown file."""
    # Extract a filename from the URL
    if url.endswith("/"):
        url = url[:-1]

    filename = url.split("/")[-1] + ".md"
    print(f"Downloading {url} into {filename}...")

    nest_asyncio.apply()

    session = HTMLSession()
    response = session.get(url, timeout=30)

    # Render the page, which will execute JavaScript
    response.html.arender()

    # Convert the rendered HTML content to markdown
    h = html2text.HTML2Text()
    markdown_content = h.handle(response.html.raw_html.decode("utf-8"))

    # Write the markdown content to a file
    filename = os.path.join(dir_path, filename)
    if not os.path.exists(filename):
        with open(filename, "w", encoding="utf-8") as f:
            f.write(markdown_content)


def download(pages: List[str]) -> str:
    """Download the HTML content from the pages and save them as markdown files."""
    # Create the content/notion directory if it doesn't exist
    # base_dir = os.path.dirname(os.path.abspath(__file__))
    dir_path = "blogs"
    os.makedirs(dir_path, exist_ok=True)
    for page in pages:
        download_and_save_in_markdown(page, dir_path)
    return dir_path


PAGES = [
    "https://quickstarts.snowflake.com/guide/data_engineering_pipelines_with_snowpark_python",
    "https://quickstarts.snowflake.com/guide/cloud_native_data_engineering_with_matillion_and_snowflake",
    "https://quickstarts.snowflake.com/guide/data_engineering_with_apache_airflow",
    "https://quickstarts.snowflake.com/guide/getting_started_with_dataengineering_ml_using_snowpark_python",
    "https://quickstarts.snowflake.com/guide/data_engineering_with_snowpark_python_and_dbt"
]

download(PAGES)

/var/folders/r0/7mcjzfsn6dg22x_bqcsmp06h0000gn/T/ipykernel_4432/2376321995.py:19: RuntimeWarning: coroutine 'HTML.arender' was never awaited
  response.html.arender()


'blogs'

In [8]:
def build_index(
                data_dir: str, 
                knowledge_base_dir: str
                ) -> None:
    """Build the vector index from the markdown files in the directory."""

    print("Building vector index...")
    documents = SimpleDirectoryReader(data_dir).load_data()

    # index = TreeIndex.from_documents(documents, service_context=service_context)
    index = VectorStoreIndex.from_documents(
                                                    documents, 
                                                    service_context=service_context, 
                                                    show_progress=True
                                                    )
    index.storage_context.persist(persist_dir=knowledge_base_dir)
    print("Done.")

In [9]:
build_index('blogs/', 'kb/')

Building vector index...


Generating embeddings: 100%|██████████| 145/145 [00:03<00:00, 38.95it/s]


Done.


In [5]:
def load_index(knowledge_base_dir: str) -> VectorStoreIndex:
    """Load the vector index from the directory."""
    print("Loading vector index...")
    storage_context = StorageContext.from_defaults(persist_dir=knowledge_base_dir)
    index = load_index_from_storage(storage_context=storage_context)
    query_engine = index.as_query_engine()
    print("Done.")
    return query_engine

In [6]:
query_engine = load_index('kb/')

Loading vector index...
INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
Done.


In [11]:
def chat_inference(
                    query: str,
                    top_k: int = 5
                    ) -> None:
    """Run a query against the index and print the results."""
    results = query_engine.query(query)
    print(f"Query: {query}")
    print(f"Response: {results}")

In [12]:
chat_inference("what data engineers are focused primarily ?")

INFO:httpx:HTTP Request: POST https://doc-ocr-adl.openai.azure.com//openai/deployments/chat_model/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
HTTP Request: POST https://doc-ocr-adl.openai.azure.com//openai/deployments/chat_model/chat/completions?api-version=2023-05-15 "HTTP/1.1 200 OK"
Query: what data engineers are focused primarily ?
Response: Data engineers are primarily focused on the data transformation phase, which involves taking source data, cleansing it, combining it, and modeling it for downstream use.
